In [34]:
from app import *

sheets = extract_sheets()
market = fetch_market(sheets['Dicts']['assets'], sheets['Others'])
forex = fetch_forex(sheets['Dicts']['depots'])
data = build_metrics(sheets['Dicts']['assets'], market, sheets['Dicts']['depots'], forex, sheets['Operation'])



[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  2 of 2 completed
 DMA
   €
 ZEN
   $
   BTC
   ESE
   GOLD
   GREENBULL
   USDT
   €


In [29]:
df = data[['DepositEUR', 'ValueEUR', 'InvestedEUR', 'CashEUR', 'PnLEUR']]
# print('\n\ndf\n', df)

df_all = df[:, 'All', 'All']
df_zen = df[:, 'ZEN', 'All']
df_dma = df[:, 'DMA', 'All']

result = pd.concat({ 
    'All' : df[:, 'All', 'All'],  
    'ZEN' : df[:, 'ZEN', 'All'],  
    'DMA' : df[:, 'DMA', 'All']
}, axis=1)

result.transpose().round(1)


InvalidIndexError: (slice(None, None, None), 'All', 'All')

In [33]:
df = data[['ValueEUR', 'InvestedEUR', 'CashEUR', 'PnLEUR', 'DepositEUR']].iloc[1]

df_all = pd.concat({ 
    'All' : df[:, 'All', 'All'],  
    'ZEN' : df[:, 'ZEN', 'All'],  
    'DMA' : df[:, 'DMA', 'All']
}, axis=1) 

df_all

,All,ZEN,DMA
ValueEUR,0.0,0.0,0.0
InvestedEUR,0.0,0.0,0.0
CashEUR,0.0,0.0,0.0
PnLEUR,0.0,0.0,0.0
DepositEUR,0.0,0.0,0.0
